# A tutorial on SDDP


# 1. first steps

Hydrothermal scheduling is the most common application of stochastic dual
dynamic programming. To illustrate some of the basic functionality of
`SDDP.jl`, we implement a very simple model of the hydrothermal scheduling
problem.

We consider the problem of scheduling electrical generation over three time
periods in order to meet a known demand of 150 MWh in each period.

There are two generators: a thermal generator, and a hydro generator. The
thermal generator has a short-run marginal cost of \\\$50/MWh in the first
stage, \\\$100/MWh in the second stage, and \\\$150/MWh in the third stage.
The hydro generator has a short-run marginal cost of \\\$0/MWh.

The hydro generator draws water from a reservoir which has a maximum capacity
of 200 units. We assume that at the start of the first time period, the
reservoir is full. In addition to the ability to generate electricity by
passing water through the hydroelectric turbine, the hydro generator can also
spill water down a spillway (bypassing the turbine) in order to prevent the
water from over-topping the dam. We assume that there is no cost of spillage.

The objective of the optimization is to minimize the expected cost of
generation over the three time periods.

## 1.1 Mathematical formulation

Let's take the problem described above and form a mathematical model. In any
multistage stochastic programming problem, we need to identify five key
features:

1. The _stages_
2. The _state_ variables
3. The _control_ variables
4. The _dynamics_
5. The _stage-objective_

#### 1.1.1 Stages

From the description, we have three stages: `t = 1, 2, 3`. Here is a picture
of what this looks like:

![Linear policy graph](assets/deterministic_linear_policy_graph.png)

Notice that the boxes form a _linear graph_. This will be important when we
get to the code. (We'll get to more complicated graphs in future tutorials.)

#### 1.1.2 State variables

State variables capture the information that flows between stages. These can
be harder to identify. However, in our model, the state variable is the volume
of water stored in the reservoir over time.

In the model below, we're going to call the state variable `volume`.

Each stage `t` is an interval in time. Thus, we need to record the value of
the state variable in each stage at two points in time: at the beginning of
the stage, which we  refer to as the _incoming_ value of the state variable;
and at the end of the  state, which we refer to as the _outgoing_ state
variable.

We're going to refer to the incoming value of `volume` by `volume.in` and the
outgoing value by `volume.out`.

Note that `volume.out` when `t=1` is equal to `volume.in` when `t=2`.

The problem description also mentions some constraints on the volume of water
in the reservoir. It cannot be negative, and the maximum level is 200 units.
Thus, we have `0 <= volume <= 200`. Also, the description says that the
initial value of water in the reservoir (i.e., `volume.in` when `t = 1`) is
200 units.

#### 1.1.3 Control variables

Control variables are the actions that the agent can take during a stage to
change the value of the state variables. (Hence the name _control_.)

There are three control variables in our problem.

1. The quantity of thermal generation, which we're going to call
   `thermal_generation`.
2. The quantity of hydro generation, which we're going to call
   `hydro_generation`.
3. The quatity of water to spill, which we're going to call `hydro_spill`.

All of these variables are non-negative.

#### 1.1.4 The dynamics

The dynamics of a problem describe how the state variables evolve through time
in response to the controls chosen by the agent.

For our problem, the state variable is the volume of water in the reservoir.
The volume of water decreases in response to water being used for hydro
generation and spillage. So the dynamics for our problem are:

`volume.out = volume.in - hydro_generation - hydro_spill`

We can also put constraints on the values of the state and control variables.
For example, in our problem, there is also a constraint that the total
generation must meet the demand of 150 MWh in each stage. So, we have a
constraint that:
`hydro_generation + thermal_generation = 150`.

 #### 1.1.5 The stage-objective

The agent's objective is to minimize the cost of generation. So in each stage,
the agent wants to minimize the quantity of thermal generation multiplied by
the short-run marginal cost of thermal generation.

In stage `t`, they want to minimize `fuel_cost[t] * thermal_generation`, where
`fuel_cost[t]` is \\\$50 when `t=1`, \\\$100 when `t=2`, and \\\$150 when
`t=3`.

We're now ready to construct a model. Since `SDDP.jl` is intended to be very
user-friendly, we're going to give the full code first, and then walk through
some of the details. However, you should be able to read through and
understand most of what is happening.

## 1.2. Creating a model

In [ ]:
using SDDP, GLPK

hydro_model1 = SDDP.LinearPolicyGraph(
    stages = 3,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = with_optimizer(GLPK.Optimizer)
) do subproblem, t
    # Define the state variable.
    @variable(subproblem, 0 <= volume <= 200, SDDP.State, initial_value = 200)
    # Define the control variables.
    @variables(subproblem, begin
        thermal_generation >= 0
        hydro_generation   >= 0
        hydro_spill        >= 0
    end)
    # Define the constraints
    @constraints(subproblem, begin
        volume.out == volume.in - hydro_generation - hydro_spill
        thermal_generation + hydro_generation == 150.0
    end)
    # Define the objective for each stage `t`. Note that we can use `t` as an
    # index for t = 1, 2, 3.
    fuel_cost = [50.0, 100.0, 150.0]
    @stageobjective(subproblem, fuel_cost[t] * thermal_generation)
end

 Wasn't that easy! Let's walk through some of the non-obvious features.

!!! tip
    For more information on `SDDP.LinearPolicyGraph`s, read
    Create a general policy graph.

#### 1.2.1 What's this weird `do` syntax?

Julia's `do` syntax looks a little weird at first, but it's just a nice way of
making a function that can be passed to another function. For example:

In [ ]:
function outer(inner::Function)
    inner(2)
end

outer() do x
    println("x^2 = ", x^2)
end

is equivalent to

In [ ]:
inner(x) = println("x^2 = ", x^2)
outer(inner)

So, in our case, we could have gone:

In [ ]:
function subproblem_builder(subproblem::JuMP.Model, t::Int)
    # Define the state variable.
    @variable(subproblem, 0 <= volume <= 200, SDDP.State, initial_value = 200)
    # Define the control variables.
    @variables(subproblem, begin
        thermal_generation >= 0
        hydro_generation   >= 0
        hydro_spill        >= 0
    end)
    # Define the constraints
    @constraints(subproblem, begin
        volume.out == volume.in - hydro_generation - hydro_spill
        thermal_generation + hydro_generation == 150.0
    end)
    # Define the objective for each stage `t`. Note that we can use `t` as an
    # index for t = 1, 2, 3.
    fuel_cost = [50.0, 100.0, 150.0]
    @stageobjective(subproblem, fuel_cost[t] * thermal_generation)
end

hydro_model1 = SDDP.LinearPolicyGraph(
    subproblem_builder,
    stages = 3,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = with_optimizer(GLPK.Optimizer)
)

#### 1.2.2 The keywords in the `SDDP.LinearPolicyGraph` constructor

Hopefully `stages` and `sense` are obvious. However, the other two are not so
clear.

`lower_bound`: you _must_ supply a valid bound on the objective. For our
problem, we know that we cannot incur a negative cost so \\\$0 is a valid lower
bound.

`optimizer`: This is borrowed directly from JuMP's `Model` constructor:
(`Model(with_optimizer(GLPK.Optimizer))`)

#### 1.2.3 Creating state variables

State variables can be created like any other JuMP variables. Think of them as
another type of variable like binary or integer. For example, to create a
binary variable in JuMP, you go:
```julia
@variable(subproblem, x, Bin)
```
whereas to create a state variable you go
```julia
@variable(subproblem, x, SDDP.State)
```

Also note that you have to pass a keyword argument called `initial_value` that
gives the incoming value of the state variable in the first stage.

#### 1.2.4 Defining the stage-objective

In a JuMP model, we can set the objective using `@objective`. For example:
```julia
@objective(subproblem, Min, fuel_cost[t] * thermal_generation)
```

Since we only need to define the objective for each stage, rather than the
whole problem, we use the `SDDP.jl`-provided `@stageobjective`.
```julia
@stageobjective(subproblem, fuel_cost[t] * thermal_generation)
```
Note that we don't have to specify the optimization sense (`Max` of `Min`)
since this is done via the `sense` keyword argument of
`SDDP.LinearPolicyGraph`.

 ## 1.3 Manipulating policies 
 
 ### 1.3.1 Training a policy

Models can be trained using the `SDDP.train` function. It accepts a
number of keyword arguments. `iteration_limit` terminates the training after
the provided number of iterations.

In [ ]:
SDDP.train(hydro_model1; iteration_limit = 10)

### 1.3.2 Saving the policy

Once you have finished training the policy, you can write the cuts to file
using `SDDP.write_cuts_to_file`. You can read these cuts into a new
model using `SDDP.read_cuts_from_file`. Note that the model must have
the same number (and names) of the state variables, as well as the same number
and names of the nodes.

You can also save the log to a CSV file using `SDDP.write_log_to_csv`.
This will create a CSV file with columns `iteration`, `simulation`, `bound`,
and `time`.

### 1.3.3 Simulating the policy

Once you have a trained policy, you can simulate it using
`SDDP.simulate`. The return value from `simulate` is a vector with one
element for each replication. Each element is itself a vector, with one
element for each stage. Each element, corresponding to a particular stage in a
particular replication, is a dictionary that records information from the
simulation.

In [ ]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    hydro_model1,
    # The number of replications.
    1,
    # A list of names to record the values of.
    [:volume, :thermal_generation, :hydro_generation, :hydro_spill]
)

replication = 1
stage = 1

simulations[replication][stage]


Ignore many of the entries for now;  they will be relevant later.

 One element of iterest is `:volume`.

In [ ]:
outgoing_volume = [stage[:volume].out for stage in simulations[1]]

 Another is `:thermal_generation`.

In [ ]:
thermal_generation = [stage[:thermal_generation] for stage in simulations[1]]

From this, we can see the optimal policy: in the first stage, use 150 MWh of
thermal generation and 0 MWh of hydro generation. In the second stage, use 100
MWh of thermal and 50 MWh of hydro. In the third and final stage, use 0 MWh of
thermal and 150 MWh of  hydro.

## 1.4 Exercise : a hot water tank problem

We consider a hot water tank for a residential building. We model the water tank as an equivalent amount of hot water at each hour $ 0 \leq s_t \leq 500$. We assume that it follows the dynamic
$$ s_0 = 200, \qquad s_{t+1} = \rho s_t + u_t - d_t $$
where $u_t \leq 200$ is the amount of warmed water between $t$ and $t+1$, for a cost $c_t$ per unit, and $d_t$ the amount of extracted water. The coefficient $0<\rho <1$ represent the heat loss.

We assume that we have the following data ($t=1$ corresponds to 4am) 

| t     ||  1    |  2   | 3    | 4    |  5   | 6    | 7    |  8   | 9    |
| ----  || ---   | ---  | ---  | ---  | ---  | ---  | ---  | ---  | ---  |
| $c_t$ ||  24.3 | 28.5 | 42.5 | 50.4 | 52.6 | 51.8 | 48.9 | 48.6 | 47.04|
| $d_t$ ||  0    | 20   | 200  | 150  | 50   | 20   | 20   | 30   | 50   |

We want to define an optimal heating strategy. Hot water after $t=9$ is considered lost.

###  Question 1 

Construct a `hw_model` representing this problem

In [ ]:
ρ = 0.9
c = [30 28.5 42.5 50.4 52.6 51.8 48.9 48.6 47.04]
d = [0    20   200  150  50   20   20   30   50]

hw_model1 = SDDP.LinearPolicyGraph(
    stages = ??? , 
    sense = :Min,
    lower_bound = ???, 
    optimizer = with_optimizer(GLPK.Optimizer)
) do subproblem, t
    # Define the state variable.
    @variable(subproblem, 0 <= s <= 500, SDDP.State, initial_value = 200)
    # Define the control variables.
    @variables(subproblem, begin
        ???
    end)
    # Define the constraints
    @constraints(subproblem, begin
        s.out == ???
    end)
    # Define the objective for each stage `t`. Note that we can use `t` as an
    # index for t = 1, 2, 3.
    @stageobjective(subproblem, ???)
end

### Question 2

Train and simulate the problem. Describe the optimal solution.

### Question 3 (Optional)

We are now going to consider a more contrained version of our problem, where $u \leq 100$. 

#### Question 3.a )
Is this new problem admissible ? Are we in a relatively complete recourse framework ?

#### Question 3.b)
Solving this new problem via SDDP won't work (try it !). 
By introducing a new variable solve the problem.





# 2. adding uncertainty

Up to now we have solved a deterministic  hydro-thermal scheduling model. We are now adding uncertainty.

Notably missing from our previous model were inflows. Inflows are the water
that flows into the reservoir through rainfall or rivers. These inflows are
uncertain, and are the cause of the main trade-off in hydro-thermal
scheduling: the desire to use water now to generate cheap electricity, against
the risk that future inflows will be low, leading to blackouts or expensive
thermal generation.

For our simple hydro model, we assume that the inflows can be modelled by a discrete
distribution with the three outcomes given in the following table:

| ω    |   0 |  50 | 100 |
| ---- | --- | --- | --- |
| P(ω) | 1/3 | 1/3 | 1/3 |

The value of the noise (the random variable) is observed by the agent at the
start of each stage. This makes the problem a _wait-and-see_ or
_hazard-decision_ formulation.

To represent this, we can draw the following picture. The wavy lines denote
the uncertainty arriving into the start of each stage (node).

![Linear policy graph](assets/stochastic_linear_policy_graph.png)

In addition to adding this uncertainty to the model, we also need to modify
the dynamics to include `inflow`:

`volume.out = volume.in + inflow - hydro_generation - hydro_spill`

## 2.1 Creating a model

To add an uncertain variable to the model, we create a new JuMP variable
`inflow`, and then call the function `SDDP.parameterize`. The
`SDDP.parameterize` function takes three arguments: the subproblem, a
vector of realizations, and a corresponding vector of probabilities.

In [ ]:
hydro_model2 = SDDP.LinearPolicyGraph(
    stages = 3,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = with_optimizer(GLPK.Optimizer)
) do subproblem, t
    # Define the state variable.
    @variable(subproblem, 0 <= volume <= 200, SDDP.State, initial_value = 200)
    # Define the control variables.
    @variables(subproblem, begin
        thermal_generation >= 0
        hydro_generation   >= 0
        hydro_spill        >= 0
        inflow
    end)
    # Define the constraints
    @constraints(subproblem, begin
        volume.out == volume.in + inflow - hydro_generation - hydro_spill
        demand_constraint, thermal_generation + hydro_generation == 150.0
    end)
    # Define the objective for each stage `t`. Note that we can use `t` as an
    # index for t = 1, 2, 3.
    fuel_cost = [50.0, 100.0, 150.0]
    @stageobjective(subproblem, fuel_cost[t] * thermal_generation)
    # Parameterize the subproblem.
    SDDP.parameterize(subproblem, [0.0, 50.0, 100.0], [1/3, 1/3, 1/3]) do ω
        JuMP.fix(inflow, ω)
    end
end

Note how we use the JuMP function
[`JuMP.fix`](http://www.juliaopt.org/JuMP.jl/v0.19/variables/#JuMP.fix) to set
the value of the `inflow` variable to `ω`.

<div class="alert alert-block alert-info">
<b>Warning :</b> `SDDP.parameterize` can only be called once in each subproblem definition!
</div>

 
    

## 2.2 Training and simulating the policy

### 2.2.1 Training the policy

As earlier we train the policy:

In [ ]:
SDDP.train(hydro_model2; iteration_limit = 50)

### 2.2.2 Simulating the policy

We can also simulate the policy. Note that this time, the simulation is
stochastic. One common approach to quantify the quality of the policy is to
perform  a Monte Carlo simulation and then form a confidence interval for the
expected cost. This confidence interval is an estimate for the upper bound.

In addition to the confidence interval, we can calculate the lower bound using
`SDDP.calculate_bound`.

In [ ]:
using Statistics

Nmc = 1000

simulations = SDDP.simulate(hydro_model2, Nmc)

objective_values = [
    sum(stage[:stage_objective] for stage in sim) for sim in simulations
]

μ = round(mean(objective_values), digits = 2)

ci = round(1.96 * std(objective_values) / sqrt(Nmc), digits = 2)

println("Confidence interval: ", μ, " ± ", ci)
println("Lower bound: ", round(SDDP.calculate_bound(hydro_model2), digits = 2))

### 2.2.3 Simulating further parameters (Optional)

In addition to simulating the primal values of variables, we can also pass
`SDDP.jl` custom recorder functions. Each of these functions takes one
argument, the JuMP subproblem, and returns anything you can compute. For
example, the dual of the demand constraint (which we named
`demand_constraint`) corresponds to the price we should charge for
electricity, since it represents the cost of each additional unit of demand.
To calculate this, we can go

In [ ]:
simulations = SDDP.simulate(
    hydro_model2,
    1,
    custom_recorders = Dict{Symbol, Function}(
        :price => (sp) -> JuMP.dual(sp[:demand_constraint])
    )
)

prices = [stage[:price] for stage in simulations[1]]

## 2.3 Exercise : Hot water with random demand

The actual demand in hot water is not known in advance. We are going to consider that, for each hour, independantly of the others, the prediction have $30\%$ chance of being $20\%$ less than predicted, $40%$ chance of being at prediction level and$30\%$ chance of being $20\%$ above prediction level.

In addition we are going to introduce a control $v_t$ which correspond to unmet demand, priced at $100$ per unit.

### Question 4.

Model, train and solve the problem with random demand. Simulate the obtained policy.

# 3. Two uncertainties

In the previous section, we created a
stochastic hydro-thermal scheduling model. In this tutorial, we extend the
problem by adding uncertainty to the fuel costs.

Previously, we assumed that the fuel cost was deterministic: \\\$50/MWh in the
first stage, \\\$100/MWh in the second stage, and \\\$150/MWh in the third
stage. For this tutorial, we assume that in addition to these base costs, the
actual fuel cost is correlated with the inflows.

Our new model for the uncertinty is given by the following table:

| ω               |   1 |   2 |    3 |
| ----            | --- | --- | ---- |
| P(ω)            | 1/3 | 1/3 |  1/3 |
| inflow          |   0 |  50 |  100 |
| fuel_multiplier | 1.5 | 1.0 | 0.75 |

In stage `t`, the objective is now to minimize:

`fuel_multiplier * fuel_cost[t] * thermal_generation`

## 3.1 Creating a model

To add an uncertain objective, we can simply call `@stageobjective`
from inside the `SDDP.parameterize` function.

In [ ]:
hydro_model3 = SDDP.LinearPolicyGraph(
    stages = 3,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = with_optimizer(GLPK.Optimizer)
) do subproblem, t
    # Define the state variable.
    @variable(subproblem, 0 <= volume <= 200, SDDP.State, initial_value = 200)
    # Define the control variables.
    @variables(subproblem, begin
        thermal_generation >= 0
        hydro_generation   >= 0
        hydro_spill        >= 0
        inflow
    end)
    # Define the constraints
    @constraints(subproblem, begin
        volume.out == volume.in + inflow - hydro_generation - hydro_spill
        thermal_generation + hydro_generation == 150.0
    end)
    fuel_cost = [50.0, 100.0, 150.0]
    # Parameterize the subproblem.
    Ω = [
        (inflow = 0.0, fuel_multiplier = 1.5),
        (inflow = 50.0, fuel_multiplier = 1.0),
        (inflow = 100.0, fuel_multiplier = 0.75)
    ]
    SDDP.parameterize(subproblem, Ω, [1/3, 1/3, 1/3]) do ω
        JuMP.fix(inflow, ω.inflow)
        @stageobjective(subproblem,
            ω.fuel_multiplier * fuel_cost[t] * thermal_generation
        )
    end
end

## 3.2 Training and simulating the policy

As in the previous two tutorials, we train and simulate the policy:

In [ ]:
SDDP.train(hydro_model3; iteration_limit = 10)


Nmc = 1000
simulations = SDDP.simulate(hydro_model3, Nmc,
[:volume, :thermal_generation, :hydro_generation, :hydro_spill])

objective_values = [
    sum(stage[:stage_objective] for stage in sim) for sim in simulations
]

using Statistics

μ = round(mean(objective_values), digits = 2)
ci = round(1.96 * std(objective_values) / sqrt(Nmc), digits = 2)

println("Confidence interval: ", μ, " ± ", ci)
println("Lower bound: ", round(SDDP.calculate_bound(hydro_model3), digits = 2))

# 4. plotting

In our previous setions, we formulated, solved, and simulated multistage
stochastic optimization problems. However, we haven't really investigated what
the solution looks like. Luckily, `SDDP.jl` includes a number of plotting
tools to help us do that. In this tutorial, we explain the tools and make some
pretty pictures.

## 4.1 Preliminaries

The next two plot types help visualize the policy. Thus, we simulate some trajectories. 

In [ ]:
Nmc = 1000
simulations = SDDP.simulate(hydro_model3, Nmc, 
    [:volume, :thermal_generation, :hydro_generation, :hydro_spill]);

Great! Now we have some data in `simulations` to visualize.

## 4.2 Spaghetti plots (Optional)

The first plotting utility we discuss is a _spaghetti_ plot (you'll understand
the name when you see the graph).

To create a spaghetti plot, begin by creating a new
`SDDP.SpaghettiPlot` instance as follows:

In [ ]:
plt = SDDP.SpaghettiPlot(simulations)

We can add plots to `plt` using the `SDDP.add_spaghetti` function.

In [ ]:
SDDP.add_spaghetti(plt; title = "Reservoir volume") do data
    return data[:volume].out
end

You don't have just return values from the simulation, you can compute things
too.

In [ ]:
SDDP.add_spaghetti(
    plt; title = "Fuel cost", ymin = 0, ymax = 250
) do data
    if data[:thermal_generation] > 0
        return data[:stage_objective] / data[:thermal_generation]
    else  # No thermal generation, so return 0.0.
        return 0.0
    end
end

Note that there are many keyword arguments in addition to `title`. For
example, we fixed the minimum and maximum values of the y-axis using `ymin`
and `ymax`. See the `SDDP.add_spaghetti` documentation for all the
arguments.

Having built the plot, we now need to display it.

In [ ]:
SDDP.save(plt, "spaghetti_plot.html", open = true)

This should open a webpage that looks like [this
one](../assets/spaghetti_plot.html).

Using the mouse, you can highlight individual trajectories by hovering over
them. This makes it possible to visualize a single trajectory across multiple
dimensions.

If you click on the plot, then trajectories that are close to the mouse
pointer are shown darker and those further away are shown lighter.

## 4.3 Publication plots

Instead of the interactive Javascript plots, you can also create some
publication ready plots using the `SDDP.publication_plot` function.

<div class="alert alert-block alert-info">
<b>Info :</b> You need to install the `Plots.jl` 
    package for this to work. We used the `GR` backend (`gr()`), but any
    `Plots.jl` backend should work.
</div>
    

`SDDP.publication_plot` implements a plot recipe to create ribbon
plots of each variable against the stages. The first argument is the vector of
simulation dictionaries and the second argument is the dictionary key that you
want to plot. Standard `Plots.jl` keyword arguments such as `title` and `xlabel`
can be used to modify the look of each plot. By default, the plot displays
ribbons of the 0-100, 10-90, and 25-75 percentiles. The dark, solid line in the
middle is the median (i.e. 50'th percentile).

In [ ]:

using Plots
plot(
    SDDP.publication_plot(simulations, title = "Outgoing volume") do data
        return data[:volume].out
    end,
    SDDP.publication_plot(simulations, title = "Thermal generation") do data
        return data[:thermal_generation]
    end,
    xlabel = "Stage",
    ylims = (0, 200),
    layout = (1, 2),
    margin_bottom = 5,
    size = (1000, 300)
)


This should open a plot window with a plot that looks like:

![publication plot](assets/publication_plot.png)

You can save this plot as a PDF using the `Plots.jl` function `savefig`:

In [ ]:
Plots.savefig("my_picture.pdf")

## 4.4 Exercise

Plot water stock, heating control and unserved demand for the hot-water tank problem.